In [124]:
import pandas as pd
import os
from pathlib import Path
import missingno as msno
import matplotlib.pyplot as plt
import itertools
from itertools import product

Several data sets need to be cleaned and merged.  School mean SAT & PSAT scores for each year are in separate .xlsx files.  Scores disaggregated by desired demographic indicators are in another set .xlsx files also separated by year.

### 2018 data wrangling

In [125]:
# Filenames & paths
# Just do 2018 data for now.  COVID affected other years

filepath_agg = '../raw_data/aggregated/'
filepath_disagg = '../raw_data/disaggregated/'

file_2017_agg = '2017 SAT PSAT District and School Overall Results_final.xlsx'
file_2018_agg = '2018 PSAT and SAT District and School Summary Achievement Results_FINAL.xlsx'


#file_2017_disagg = '2017 SAT PSAT10 Disaggregated Report Formatted.xlsx'
file_2018_disagg = '2018 PSAT and SAT State Achievement Results Disaggregated by Subgroups.xlsx'

In [126]:
###  Longest runtime ###

# Files all have different formats

# Agg data is all on one sheet in xlsx file.
df_2017_agg_raw = pd.read_excel(filepath_agg + file_2017_agg)
df_2018_agg_raw = pd.read_excel(filepath_agg + file_2018_agg)

#df_2017_disagg_raw = pd.read_excel(filepath_disagg + file_2017_disagg)
# Disaggregated data for 2018 is split into separate sheets in xlsx file.
# Create dict of df's for each set of disagg data
dict_2018_disagg_raw = pd.read_excel(filepath_disagg + file_2018_disagg, sheet_name=None)

## Aggregated data only

In [127]:
# Drop rows with descriptive text
df_2017_agg = df_2017_agg_raw.drop(df_2017_agg_raw.index[0:4])
df_2018_agg = df_2018_agg_raw.drop(df_2018_agg_raw.index[0:3])

In [128]:
# Set columns headers as first row containing Test, District Number, District Name, etc.
df_2017_agg.columns = df_2017_agg.iloc[0]
df_2018_agg.columns = df_2018_agg.iloc[0]

In [129]:
# Drop first row containing the column headers
df_2017_agg = df_2017_agg.drop(df_2017_agg.index[0])
df_2018_agg = df_2018_agg.drop(df_2018_agg.index[0])

In [130]:
## 2017 data
# Assign state & district results to their own dataframes respectively (if they exist)
state_2017_agg = df_2017_agg.loc[df_2017_agg['School Name'] == 'STATE RESULTS']
district_2017_agg = df_2017_agg.loc[df_2017_agg['School Name'] == 'DISTRICT RESULTS']

# Drop those & create new dataframe of school only data
# Note: There's something weird with the school districts here.  BOCES?
schools_2017_agg = df_2017_agg[(df_2017_agg['School Name'] != 'STATE RESULTS') & (df_2017_agg['School Name'] != 'DISTRICT RESULTS')]


## 2018 data
# Assign state & district results to their own dataframes respectively (if they exist)
state_2018_agg = df_2018_agg.loc[df_2018_agg['Level'] == 'STATE']
district_2018_agg = df_2018_agg.loc[df_2018_agg['Level'] == 'DISTRICT']

# Create dataframe with only school level scores
schools_2018_agg = df_2018_agg.loc[df_2018_agg['Level'] == 'SCHOOL']

# Drop some unneeded columns
schools_2018_agg = schools_2018_agg.drop(['Level','Grade'], axis = 1)

In [131]:
# Rename and reindex 2017 and 2018 data

# Rename some columns for convenience
col_names1_long = list(schools_2017_agg.columns)
col_names1_short = ['Test',
                 'District Number',
                 'District Name',
                 'School Number',
                 'School Name',
                 'Total Students',
                 '2017 Valid Scores',
                 '2017 EBRW Mean',
                 '2017 Math Mean',
                 '2017 Overall Mean',
                 '2017 Participation Percent',
                 '2016 Valid Scores',
                 '2016 EBRW Mean',
                 '2016 Math Mean',
                 '2016 Overall Mean',
                 '2016 Participation Percent',
                 'Mean Overall Score Change']
schools_2017_agg = schools_2017_agg.rename(columns = dict(zip(col_names1_long, col_names1_short)))

# Rename some columns for convenience
col_names2_long = list(schools_2018_agg.columns)
col_names2_short = ['Test',
                 'District Number',
                 'District Name',
                 'School Number',
                 'School Name',
                 'Total Students',
                 '2018 Valid Scores',
                 '2018 EBRW Mean',
                 '2018 Math Mean',
                 '2018 Overall Mean Score',
                 '2018 Participation Percent',
                 '2017 Valid Scores',
                 '2017 EBRW Mean',
                 '2017 Math Mean',
                 '2017 Overall Mean Score',
                 '2017 Participation Percent',
                 'Mean Overall Score Change']
schools_2018_agg = schools_2018_agg.rename(columns = dict(zip(col_names2_long, col_names2_short)))

# Set index to school number
#schools_2017_agg['School Number'] = schools_2017_agg['School Number'].astype(int)
#schools_2017_agg = schools_2017_agg.set_index('School Number')

#schools_2018_agg['School Number'] = schools_2018_agg['School Number'].astype(int)
#schools_2018_agg = schools_2018_agg.set_index('School Number')

In [132]:
# Trim white space
schools_2017_agg['Test'] = schools_2017_agg['Test'].str.strip()
schools_2018_agg['Test'] = schools_2018_agg['Test'].str.strip()

#### Create new dataframe aggregating all data from 2018 into a cleaner format

In [133]:
# Select only rows that have both 2017 and 2018 scores
schools_2018_agg_subset = schools_2018_agg[~schools_2018_agg['Mean Overall Score Change'].isna()]

# Drop rows using '*' to indicate missing data
schools_2018_agg_subset = schools_2018_agg_subset[~(schools_2018_agg_subset['Total Students'] == '*')]

In [134]:
###  May not be necessary ###

# Set appropriate names for each column

score_types = ['Valid Scores',
              'EBRW Mean',
              'Math Mean',
              'Overall Mean Score',
              'Participation Percent']
years_str = ['2017', '2018']
test_types = ['PSAT10', 'SAT']

score_cols = list(product(years_str, test_types, score_types))
score_cols = [year + " " + test + " " + score for year, test, score in score_cols]

all_cols = ['School Number', 'District Name', 'School Name'] + score_cols

In [135]:
schools_2018_agg_SAT = schools_2018_agg_subset.loc[schools_2018_agg_subset["Test"] == "SAT"]
schools_2018_agg_PSAT = schools_2018_agg_subset.loc[schools_2018_agg_subset["Test"] == "PSAT10"]
tests_combined_2018_agg = schools_2018_agg_SAT.merge(schools_2018_agg_PSAT, on="School Number", suffixes=(' SAT', ' PSAT10'))
#list(tests_combined_2018.columns)

In [136]:
# Drop duplicated &  unnecessary columns
tests_combined_2018_agg = tests_combined_2018_agg.drop(['Test PSAT10',
                                                        'Test SAT',
                                                'District Number PSAT10',
                                                'District Name PSAT10',
                                                'School Name PSAT10',
                                                        'Test PSAT10',
                                                        'Mean Overall Score Change SAT',
                                                        'Mean Overall Score Change PSAT10'
                                               ], axis = 1)
#list(tests_combined_2018_agg.columns)

In [137]:
# Fix column names with unnecessary suffix
tests_combined_2018_agg = tests_combined_2018_agg.rename(columns={'District Number SAT': "District Number", "District Name SAT": "District Name", "School Name SAT": "School Name"}, errors="raise")

In [138]:
tests_combined_2018_agg.head()

3,District Number,District Name,School Number,School Name,Total Students SAT,2018 Valid Scores SAT,2018 EBRW Mean SAT,2018 Math Mean SAT,2018 Overall Mean Score SAT,2018 Participation Percent SAT,...,2018 Valid Scores PSAT10,2018 EBRW Mean PSAT10,2018 Math Mean PSAT10,2018 Overall Mean Score PSAT10,2018 Participation Percent PSAT10,2017 Valid Scores PSAT10,2017 EBRW Mean PSAT10,2017 Math Mean PSAT10,2017 Overall Mean Score PSAT10,2017 Participation Percent PSAT10
0,0010,MAPLETON 1,0187,MAPLETON EXPEDITIONARY SCHOOL OF THE ARTS,112,99,472,467,939,88.4,...,116,431,420,851,91.3,88,412,430,842,83.8
1,0010,MAPLETON 1,0212,MAPLETON EARLY COLLEGE HIGH SCHOOL,65,62,464,441,905,95.4,...,59,430,410,840,96.7,60,409,409,818,96.8
2,0010,MAPLETON 1,0263,GLOBAL LEADERSHIP ACADEMY,42,39,443,438,880,92.9,...,65,431,394,825,100.0,50,410,404,814,92.6
3,0010,MAPLETON 1,0309,ACADEMY HIGH SCHOOL,109,107,452,434,886,98.2,...,121,407,398,805,97.6,117,411,417,828,96.7
4,0010,MAPLETON 1,0503,YORK INTERNATIONAL,55,51,495,477,972,92.7,...,73,439,434,873,93.6,62,466,444,910,91.2


Next steps?
* Keep only 2018 SAT and 2017 PSAT scores?

## ---Done with aggregated data---

# Data disaggretaed by subgroups

In [139]:
# Rename dict_2018_disagg_raw keys
disagg_categories = ['Gend', 'Ethn', 'FRM', 'GT','ELL','Migr', 'IEP']
dict_2018_disagg = dict(zip(disagg_categories, list(dict_2018_disagg_raw.values())))

In [162]:
# Function to clean each sheet

def clean_disagg_sheet(sheet_raw):
    
    # Drop rows with descriptive text
    sheet = sheet_raw.drop(sheet_raw.index[0:4])
    
    # Set columns headers as first row containing Test, District Number, District Name, etc.
    sheet.columns = sheet.iloc[0]

    # Drop first row containing the column headers
    sheet = sheet.drop(sheet.index[0])
    
    #Rename some columns for convenience
    col_name_long = list(sheet.columns)
    col_names_short = ['Level',
                     'Test',
                     'District Number',
                     'District Name',
                     'School Number',
                     'School Name',
                     'Demographic Group',
                     '2018 Total Records',
                     '2018 Valid Scores',
                     '2018 Participation Rate',
                     '2018 EBRW Mean',
                     '2018 Math Mean',
                     '2018 Overall Mean Score']
    sheet = sheet.rename(columns = dict(zip(col_name_long, col_names_short)))
    
    # Drop unnecessary & missing values
    sheet = sheet[sheet['Test'] == 'SAT']
    sheet = sheet[sheet['2018 Valid Scores'] != '< 16']
    sheet = sheet[sheet['2018 Overall Mean Score'] != '*']
    sheet = sheet[sheet['Demographic Group'] != 'Not Reported']
    
    # Drop unnecessary columns
    sheet = sheet.drop(['Test',
                      '2018 Total Records',
                      '2018 Participation Rate',
                      '2018 Valid Scores',
                       'District Name',
                       'School Name'], axis=1)
    
    # Assign state & district results to their own dataframes respectively (if they exist)
    state_sheet = sheet.loc[sheet['Level'] == 'STATE']
    district_sheet = sheet.loc[sheet['Level'] == 'DISTRICT']

    state_sheet = state_sheet.drop(['Level'], axis=1)
    district_sheet = district_sheet.drop(['Level'], axis=1)
    
    # Drop those & create new dataframe of school only data
    schools_sheet = sheet[sheet['Level'] == 'SCHOOL']
    schools_sheet = schools_sheet.drop(['Level'], axis=1)
    
       
    return schools_sheet, district_sheet, state_sheet

In [165]:
gend_sheets = clean_disagg_sheet(dict_2018_disagg['Gend'])
gend_sheets[0]

4,District Number,School Number,Demographic Group,2018 EBRW Mean,2018 Math Mean,2018 Overall Mean Score
3048,0010,0212,Female,472,438,910
3049,0010,1796,Female,496,444,940
3050,0010,1796,Male,504,480,984
3051,0010,0187,Male,467,474,941
3052,0010,0212,Male,454,444,898
...,...,...,...,...,...,...
4044,8001,6914,Male,481,482,962
4045,8001,8825,Female,637,592,1229
4046,8001,6914,Female,468,450,918
4047,8001,0015,Female,526,486,1012
